In [1]:
!pip install tensorflow keras nltk

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [42]:
import requests

url = 'https://drive.google.com/uc?id=1q5glYZgRtmUNFJw8aBvpv5i2YeuJEDwi'
response = requests.get(url)
with open('expanded_medical.json', 'wb') as f:
    f.write(response.content)

In [30]:
with open('expanded_medical.json', 'r') as file:
    data = json.load(file)

In [ ]:
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))


In [ ]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)


for i, t in enumerate(training):
    print(f"Training sample {i}: bag shape = {len(t[0])}, output_row shape = {len(t[1])}")
    if len(t[0]) != len(training[0][0]) or len(t[1]) != len(training[0][1]):
       print(f"Inconsistent shapes at sample {i}")
       break


training = np.array(training, dtype='object')
train_x = np.array([t[0] for t in training])
train_y = np.array([t[1] for t in training])

Streaming output truncated to the last 5000 lines.
Training sample 13199: bag shape = 159, output_row shape = 55
Training sample 13200: bag shape = 159, output_row shape = 55
Training sample 13201: bag shape = 159, output_row shape = 55
Training sample 13202: bag shape = 159, output_row shape = 55
Training sample 13203: bag shape = 159, output_row shape = 55
Training sample 13204: bag shape = 159, output_row shape = 55
Training sample 13205: bag shape = 159, output_row shape = 55
Training sample 13206: bag shape = 159, output_row shape = 55
Training sample 13207: bag shape = 159, output_row shape = 55
Training sample 13208: bag shape = 159, output_row shape = 55
Training sample 13209: bag shape = 159, output_row shape = 55
Training sample 13210: bag shape = 159, output_row shape = 55
Training sample 13211: bag shape = 159, output_row shape = 55
Training sample 13212: bag shape = 159, output_row shape = 55
Training sample 13213: bag shape = 159, output_row shape = 55
Training sample 132

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5")

Epoch 1/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8462 - loss: 0.5945
Epoch 2/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9878 - loss: 0.0585
Epoch 3/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9864 - loss: 0.0593
Epoch 4/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9882 - loss: 0.0520
Epoch 5/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9890 - loss: 0.0458
Epoch 6/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9875 - loss: 0.0532
Epoch 7/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9892 - loss: 0.0439
Epoch 8/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9894 - loss: 0.0442
Epoch 9/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9882 - loss: 0.0477
Epoch 10/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9895 - loss: 0.0418
Epoch 11/200
3640/3640 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9882 - loss: 0.0470
Epoch 12/200
3640

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def get_response(intent, intents_json):
    for i in intents_json['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])

In [ ]:
while True:
    message = input("You: ")
    intents = predict_class(message)
    if intents:
        response = get_response(intents[0]['intent'], data)
        print(f"Bot: {response}")
    else:
        print("Bot: Sorry, I didn't understand that.")

You: hi i have a cold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bot: 1)Keeping hydrated is absolutely vital to help 'flush' out the cold, as well as to break down congestion and keep your throat lubricated. 2)Vitamin C is extremely helpful when fighting infection, so at the first sign of a cold be sure to increase your intake by eating plenty of berries, citrus fruits, papayas, broccoli and red peppers which will help keep you protected. 3)When it comes to combating a cold,Vitamin D is essential in helping to regulate immune response.
